In [34]:
import pandas as pd
# pd.set_option('max_colwidth', 500)

siads_csv = pd.read_csv('SIADS_RelatorioBalancoFisicoFinanceiro.csv', encoding='utf-8')


uasg = ''
uasg_nome = ''
ug = ''
ug_nome = ''
ourg = ''
# siads_csv.to_clipboard(index=False)

# siads_csv.to_clipboard(index=False)
# display(siads_csv)

print(len(siads_csv))
for i in range(0, len(siads_csv)):
    if pd.notna(siads_csv['Unnamed: 0'][i]):
        print(siads_csv['Unnamed: 0'][i])
        siads_csv['Unnamed: 1'][i] = siads_csv['Unnamed: 0'][i]
    if pd.notna(siads_csv['Unnamed: 7'][i]) and (siads_csv['Unnamed: 7'][i] != '0.00'):
        siads_csv['Unnamed: 1'][i] = siads_csv['Unnamed: 7'][i]

siads_csv.to_clipboard(index=False)

# Remover valores NaN da coluna 'Unnamed: 0' antes de aplicar .str.contains()
# ug_info = siads_csv[siads_csv['MATERIAL:'].notna()]
# ug_info = ug_info['26418 - INST.FED.DE EDUC.,CIENC.E TEC.DE PERNAMBUCO']
# ug_info = ug_info[ug_info.notna()]
# ug_info = ug_info.drop_duplicates()
# ug_info = ug_info.rename("infos")
# # ug_info = ug_info[ug_info['26418 - INST.FED.DE EDUC.,CIENC.E TEC.DE PERNAMBUCO'].str.contains('-')]
# # ug_info.to_clipboard(index=False)
# uasg = ug_info.iloc[0].split('-')
# uasg_nome = uasg[1].strip()
# uasg = uasg[0].strip()

# ug = ug_info.iloc[1].split('-')
# ug_nome = ug[1].strip()
# ug = ug[0].strip()
# # print(uasg, uasg_nome, ug, ug_nome)
# # uasg = ug_info['26418 - INST.FED.DE EDUC.,CIENC.E TEC.DE PERNAMBUCO']

# # Aplicar o filtro somente nas linhas que contém '-'
# siads_csv = siads_csv[['Unnamed: 0', 'Unnamed: 12', 'Unnamed: 14']]

# siads_csv = siads_csv[siads_csv['Unnamed: 0'].notna()]
# siads_csv = siads_csv[siads_csv['Unnamed: 0'].str.contains('-')]
# siads_csv = siads_csv.drop_duplicates()
# # print(siads_csv['MATERIAL:'].str.contains('-'))

# display(siads_csv)
# print(len(siads_csv))

2210
CONTA
115610100 - MATERIAIS DE CONSUMO
07 - GENEROS DE ALIMENTACAO
TOTAL  - 07 - GENEROS DE ALIMENTACAO (R$)
14 - MATERIAL EDUCATIVO E ESPORTIVO
EMITIDO POR:
CONTA
115610100 - MATERIAIS DE CONSUMO
14 - MATERIAL EDUCATIVO E ESPORTIVO
EMITIDO POR:
CONTA
115610100 - MATERIAIS DE CONSUMO
14 - MATERIAL EDUCATIVO E ESPORTIVO
EMITIDO POR:
CONTA
115610100 - MATERIAIS DE CONSUMO
14 - MATERIAL EDUCATIVO E ESPORTIVO
EMITIDO POR:
CONTA
115610100 - MATERIAIS DE CONSUMO
14 - MATERIAL EDUCATIVO E ESPORTIVO
EMITIDO POR:
CONTA
115610100 - MATERIAIS DE CONSUMO
14 - MATERIAL EDUCATIVO E ESPORTIVO
EMITIDO POR:
CONTA
115610100 - MATERIAIS DE CONSUMO
14 - MATERIAL EDUCATIVO E ESPORTIVO
EMITIDO POR:
CONTA
115610100 - MATERIAIS DE CONSUMO
14 - MATERIAL EDUCATIVO E ESPORTIVO
EMITIDO POR:
CONTA
115610100 - MATERIAIS DE CONSUMO
14 - MATERIAL EDUCATIVO E ESPORTIVO
EMITIDO POR:
CONTA
115610100 - MATERIAIS DE CONSUMO
14 - MATERIAL EDUCATIVO E ESPORTIVO
EMITIDO POR:
CONTA
115610100 - MATERIAIS DE CONSUMO
14 - M

C:\Users\Ricar\AppData\Local\Temp\ipykernel_3068\602520417.py:21: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  siads_csv['Unnamed: 1'][i] = siads_csv['Unnamed: 0'][i]
C:\Users\Ricar\AppData\Local\Temp\ipykernel_3068\602520417.py:21: Setting

16 - MATERIAL DE EXPEDIENTE
EMITIDO POR:
CONTA
115610100 - MATERIAIS DE CONSUMO
16 - MATERIAL DE EXPEDIENTE
EMITIDO POR:
CONTA
115610100 - MATERIAIS DE CONSUMO
16 - MATERIAL DE EXPEDIENTE
EMITIDO POR:
CONTA
115610100 - MATERIAIS DE CONSUMO
16 - MATERIAL DE EXPEDIENTE
EMITIDO POR:
CONTA
115610100 - MATERIAIS DE CONSUMO
16 - MATERIAL DE EXPEDIENTE
EMITIDO POR:
CONTA
115610100 - MATERIAIS DE CONSUMO
16 - MATERIAL DE EXPEDIENTE
EMITIDO POR:
CONTA
115610100 - MATERIAIS DE CONSUMO
16 - MATERIAL DE EXPEDIENTE
EMITIDO POR:
CONTA
115610100 - MATERIAIS DE CONSUMO
16 - MATERIAL DE EXPEDIENTE
EMITIDO POR:
CONTA
115610100 - MATERIAIS DE CONSUMO
16 - MATERIAL DE EXPEDIENTE
EMITIDO POR:
CONTA
115610100 - MATERIAIS DE CONSUMO
16 - MATERIAL DE EXPEDIENTE
EMITIDO POR:
CONTA
115610100 - MATERIAIS DE CONSUMO
16 - MATERIAL DE EXPEDIENTE
EMITIDO POR:
CONTA
115610100 - MATERIAIS DE CONSUMO
16 - MATERIAL DE EXPEDIENTE
EMITIDO POR:
CONTA
115610100 - MATERIAIS DE CONSUMO
16 - MATERIAL DE EXPEDIENTE
EMITIDO POR:

C:\Users\Ricar\AppData\Local\Temp\ipykernel_3068\602520417.py:21: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  siads_csv['Unnamed: 1'][i] = siads_csv['Unnamed: 0'][i]
C:\Users\Ricar\AppData\Local\Temp\ipykernel_3068\602520417.py:21: Setting

TOTAL  - 21 - MATERIAL DE CONSUMO (R$)
22 - MATERIAL DE CONSUMO
EMITIDO POR:
CONTA
115610100 - MATERIAIS DE CONSUMO
22 - MATERIAL DE CONSUMO
EMITIDO POR:
CONTA
115610100 - MATERIAIS DE CONSUMO
22 - MATERIAL DE CONSUMO
EMITIDO POR:
CONTA
115610100 - MATERIAIS DE CONSUMO
22 - MATERIAL DE CONSUMO
EMITIDO POR:
CONTA
115610100 - MATERIAIS DE CONSUMO
22 - MATERIAL DE CONSUMO
TOTAL  - 22 - MATERIAL DE CONSUMO (R$)
23 - UNIFORMES, TECIDOS E AVIAMENTOS
EMITIDO POR:
CONTA
115610100 - MATERIAIS DE CONSUMO
23 - UNIFORMES, TECIDOS E AVIAMENTOS
TOTAL  - 23 - UNIFORMES, TECIDOS E AVIAMENTOS (R$)
24 - MATERIAL P/ MANUT.DE BENS IMOVEIS/INSTALACOES
EMITIDO POR:
CONTA
115610100 - MATERIAIS DE CONSUMO
24 - MATERIAL P/ MANUT.DE BENS IMOVEIS/INSTALACOES
EMITIDO POR:
CONTA
115610100 - MATERIAIS DE CONSUMO
24 - MATERIAL P/ MANUT.DE BENS IMOVEIS/INSTALACOES
EMITIDO POR:
CONTA
115610100 - MATERIAIS DE CONSUMO
24 - MATERIAL P/ MANUT.DE BENS IMOVEIS/INSTALACOES
EMITIDO POR:
CONTA
115610100 - MATERIAIS DE CONSUMO


C:\Users\Ricar\AppData\Local\Temp\ipykernel_3068\602520417.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  siads_csv['Unnamed: 1'][i] = siads_csv['Unnamed: 7'][i]
C:\Users\Ricar\AppData\Local\Temp\ipykernel_3068\602520417.py:23: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform

EMITIDO POR:
CONTA
115610100 - MATERIAIS DE CONSUMO
24 - MATERIAL P/ MANUT.DE BENS IMOVEIS/INSTALACOES
EMITIDO POR:
CONTA
115610100 - MATERIAIS DE CONSUMO
24 - MATERIAL P/ MANUT.DE BENS IMOVEIS/INSTALACOES
EMITIDO POR:
CONTA
115610100 - MATERIAIS DE CONSUMO
24 - MATERIAL P/ MANUT.DE BENS IMOVEIS/INSTALACOES
EMITIDO POR:
CONTA
115610100 - MATERIAIS DE CONSUMO
24 - MATERIAL P/ MANUT.DE BENS IMOVEIS/INSTALACOES
EMITIDO POR:
CONTA
115610100 - MATERIAIS DE CONSUMO
24 - MATERIAL P/ MANUT.DE BENS IMOVEIS/INSTALACOES
TOTAL  - 24 - MATERIAL P/ MANUT.DE BENS IMOVEIS/INSTALACOES (R$)
25 - MATERIAL P/ MANUTENCAO DE BENS MOVEIS
TOTAL  - 25 - MATERIAL P/ MANUTENCAO DE BENS MOVEIS (R$)
26 - MATERIAL ELETRICO E ELETRONICO
EMITIDO POR:
CONTA
115610100 - MATERIAIS DE CONSUMO
26 - MATERIAL ELETRICO E ELETRONICO
EMITIDO POR:
CONTA
115610100 - MATERIAIS DE CONSUMO
26 - MATERIAL ELETRICO E ELETRONICO
EMITIDO POR:
CONTA
115610100 - MATERIAIS DE CONSUMO
26 - MATERIAL ELETRICO E ELETRONICO
EMITIDO POR:
CONTA
1

In [2]:
siads_df = pd.DataFrame(columns=['uasg', 'uasg_nome', 'ug', 'ug_nome', 'conta', 'conta_nome', 'subitem', 'subitem_nome', 'uorg', 
                                 'codigo', 'descricao', 'qtd', 'valor', 'valor_total'])
new_row = {
    'uasg':uasg,
    'uasg_nome':uasg_nome,
    'ug':ug,
    'ug_nome':ug_nome,
    'conta':'',
    'conta_nome':'',
    'subitem':'',
    'subitem_nome':'',
    'uorg':'',
    'codigo':'',
    'descricao':'',
    'qtd':0,
    'valor':0.0,
    'valor_total': 0.0
}

# display(siads_df) 223 218 135
# content = str(siads_csv.iloc[32])
# print(content)
# content = content.split('-')
# content = content[-1].strip().split('\n')

# qqtd = content[0].split()
# qqtd = qqtd[-1]

# vlerr = content[1].split()
# vlerr = vlerr[-1]

# print('tamanho content', len(content))
# print(content)
# print('qtd', qqtd)
# print('valor', vlerr)

In [3]:
# for i in range(0, 40):
for i in range(0, len(siads_csv)):
    content = str(siads_csv.iloc[i])
    content = content.split('-')
    if 'CONTA:' in content[0]:        
        new_row['conta'] = content[0]
        new_row['conta'] = new_row['conta'].split()
        new_row['conta'] = new_row['conta'][-1]
    
        new_row['conta_nome'] = content[1].strip().split('\n')
        new_row['conta_nome'] = new_row['conta_nome'][0]
    
    if 'SUBITEM:' in content[0]:
        new_row['subitem'] = content[0]
        new_row['subitem'] = content[0].split()
        new_row['subitem'] = new_row['subitem'][-1]

        new_row['subitem_nome'] = content[1].split('-')
        new_row['subitem_nome'] = content[1].strip().split('\n')
        new_row['subitem_nome'] = new_row['subitem_nome'][0]

    try:
        new_row['uorg'] = content[0].strip().split()
        new_row['uorg'] = new_row['uorg'][-1]
        
        new_row['codigo'] = content[1].strip()
        
        new_row['descricao'] = content[2].strip().split('\n')
        new_row['descricao'] = new_row['descricao'][0]
        
        
        if len(content) == 3:
            print("tamanho content", len(content))
            
            new_row['qtd'] = content[-1].strip().split('\n')
            new_row['qtd'] = new_row['qtd'][1].split()
            new_row['qtd'] = new_row['qtd'][-1]
            
            # new_row['qtd'] = new_row['qtd'].replace('.', '')
            # new_row['qtd'] = new_row['qtd'].replace(',', '.')
            new_row['qtd'] = int(float(new_row['qtd']))
    
            new_row['valor'] = content[-1].strip().split('\n')
            new_row['valor'] = new_row['valor'][2].split()
            new_row['valor'] = new_row['valor'][-1]
            
            new_row['valor'] = new_row['valor'].replace('.', '')
            new_row['valor'] = new_row['valor'].replace(',', '.')
            new_row['valor'] = float(new_row['valor'])
        
        else:
        
            new_row['qtd'] = content[-1].strip().split('\n')
            new_row['qtd'] = new_row['qtd'][1].split()
            new_row['qtd'] = new_row['qtd'][-1]
            
            # new_row['qtd'] = new_row['qtd'].replace('.', '')
            # new_row['qtd'] = new_row['qtd'].replace(',', '.')
            new_row['qtd'] = int(float(new_row['qtd']))
    
            new_row['valor'] = content[-1].strip().split('\n')
            new_row['valor'] = new_row['valor'][2].split()
            new_row['valor'] = new_row['valor'][-1]

            new_row['valor'] = new_row['valor'].replace('.', '')
            new_row['valor'] = new_row['valor'].replace(',', '.')
            new_row['valor'] = float(new_row['valor'])
        
        new_row['valor_total'] = new_row['valor'] * new_row['qtd'] 
        new_row['valor_total'] = round(new_row['valor_total'], 2)
    
    except(IndexError, ValueError):
        continue
    
    # print(siads_csv.iloc[0])
    siads_df.loc[len(siads_df)] = new_row

tamanho content 3
tamanho content 3
tamanho content 3
tamanho content 3
tamanho content 3
tamanho content 3
tamanho content 3
tamanho content 3
tamanho content 3
tamanho content 3
tamanho content 3
tamanho content 3
tamanho content 3
tamanho content 3
tamanho content 3
tamanho content 3
tamanho content 3
tamanho content 3
tamanho content 3
tamanho content 3
tamanho content 3
tamanho content 3
tamanho content 3
tamanho content 3
tamanho content 3
tamanho content 3
tamanho content 3
tamanho content 3
tamanho content 3
tamanho content 3
tamanho content 3
tamanho content 3
tamanho content 3
tamanho content 3
tamanho content 3
tamanho content 3
tamanho content 3
tamanho content 3
tamanho content 3
tamanho content 3
tamanho content 3
tamanho content 3
tamanho content 3
tamanho content 3
tamanho content 3
tamanho content 3
tamanho content 3
tamanho content 3
tamanho content 3
tamanho content 3
tamanho content 3
tamanho content 3
tamanho content 3
tamanho content 3
tamanho content 3
tamanho co

In [4]:
display(siads_df)
name = 'cafi'
print(siads_df.info())
# siads_df.to_clipboard(index=False)
siads_df.to_excel(f'{name}_siads_inventario.xlsx', index=False)

,uasg,uasg_nome,ug,ug_nome,conta,conta_nome,subitem,subitem_nome,uorg,codigo,descricao,qtd,valor,valor_total
0,151911,IFPE,26418,"INST.FED.DE EDUC.,CIENC.E TEC.DE PERNAMBUCO",115610100,MATERIAIS DE CONSUMO,14,MATERIAL EDUCATIVO E ESPORTIVO,74691,000424218,MEDALHA EM BRONZE COMEMORA...,19,1.80,34.20
1,151911,IFPE,26418,"INST.FED.DE EDUC.,CIENC.E TEC.DE PERNAMBUCO",115610100,MATERIAIS DE CONSUMO,14,MATERIAL EDUCATIVO E ESPORTIVO,74691,000503231,CRONOMETRO DIGITAL RESISTE...,5,71.85,359.25
2,151911,IFPE,26418,"INST.FED.DE EDUC.,CIENC.E TEC.DE PERNAMBUCO",115610100,MATERIAIS DE CONSUMO,14,MATERIAL EDUCATIVO E ESPORTIVO,74691,000819808,"APITO EM PLáSTICO, PROFISS...",11,12.87,141.57
3,151911,IFPE,26418,"INST.FED.DE EDUC.,CIENC.E TEC.DE PERNAMBUCO",115610100,MATERIAIS DE CONSUMO,14,MATERIAL EDUCATIVO E ESPORTIVO,74691,000926469,"PRANCHETA TATICA,MAGNETICA...",1,82.00,82.00
4,151911,IFPE,26418,"INST.FED.DE EDUC.,CIENC.E TEC.DE PERNAMBUCO",115610100,MATERIAIS DE CONSUMO,14,MATERIAL EDUCATIVO E ESPORTIVO,74691,001044176,CANELEIRA PARA FUTSAL,3,9.50,28.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,151911,IFPE,26418,"INST.FED.DE EDUC.,CIENC.E TEC.DE PERNAMBUCO",115610100,MATERIAIS DE CONSUMO,59,MATERIAL PARA DIVULGACAO,74691,001727206,ESTOJO CONFECCIONADO EM LO...,500,7.00,3500.00
361,151911,IFPE,26418,"INST.FED.DE EDUC.,CIENC.E TEC.DE PERNAMBUCO",115610100,MATERIAIS DE CONSUMO,59,MATERIAL PARA DIVULGACAO,74691,001744844,CANETA ECOLOGICA PERSONALI...,890,3.27,2910.30
362,151911,IFPE,26418,"INST.FED.DE EDUC.,CIENC.E TEC.DE PERNAMBUCO",115610100,MATERIAIS DE CONSUMO,59,MATERIAL PARA DIVULGACAO,74691,001978160,"AGENDA PLANNER, FORMATO FE...",385,26.60,10241.00
363,151911,IFPE,26418,"INST.FED.DE EDUC.,CIENC.E TEC.DE PERNAMBUCO",115610100,MATERIAIS DE CONSUMO,59,MATERIAL PARA DIVULGACAO,74691,001978225,BLOCO DE ANOTAÇÕES INSTITU...,378,3.45,1304.10


<class 'pandas.core.frame.DataFrame'>
Index: 365 entries, 0 to 364
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   uasg          365 non-null    object 
 1   uasg_nome     365 non-null    object 
 2   ug            365 non-null    object 
 3   ug_nome       365 non-null    object 
 4   conta         365 non-null    object 
 5   conta_nome    365 non-null    object 
 6   subitem       365 non-null    object 
 7   subitem_nome  365 non-null    object 
 8   uorg          365 non-null    object 
 9   codigo        365 non-null    object 
 10  descricao     365 non-null    object 
 11  qtd           365 non-null    int64  
 12  valor         365 non-null    float64
 13  valor_total   365 non-null    float64
dtypes: float64(2), int64(1), object(11)
memory usage: 42.8+ KB
None


In [5]:
totalizacao = siads_df['valor_total'].sum()
print(totalizacao)

176340.71000000002


In [15]:
subitem_total = pd.DataFrame()
subitem_total['subitem_nome'] = siads_df['subitem_nome'].drop_duplicates().reset_index(drop=True)
somatorio = []
sub_siads_one = siads_df[siads_df['subitem_nome'].str.contains(f'{subitem_total['subitem_nome'][3]}')]
# sub_siads_one.to_clipboard()
# display(sub_siads)
# print(subitem_total['subitem_nome'][0])
for i in range(0, len(subitem_total)):
    sub_siads = siads_df[siads_df['subitem_nome'].str.contains(f'{subitem_total['subitem_nome'][i]}')]
    soma = sub_siads['valor_total'].sum()
    somatorio.append(soma)
    print(subitem_total['subitem_nome'][i])
    # print(subitem_total.iloc[i])
# display(subitem_total)
subitem_total['total'] = somatorio
display(subitem_total)

MATERIAL EDUCATIVO E ESPORTIVO
MATERIAL DE EXPEDIENTE
MATERIAL DE TIC
MATERIAL DE ACONDICIONAMENTO E E...
MATERIAL DE COPA E COZINHA
MATERIAL DE LIMPEZA E PROD. DE
MATERIAL P/ MANUT.DE BENS IMOV
MATERIAL ELETRICO E ELETRONICO
MATERIAL DE PROTECAO E SEGURANCA
FERRAMENTAS
MATERIAL PARA DIVULGACAO


,subitem_nome,total
0,MATERIAL EDUCATIVO E ESPORTIVO,25188.59
1,MATERIAL DE EXPEDIENTE,48996.35
2,MATERIAL DE TIC,5622.05
3,MATERIAL DE ACONDICIONAMENTO E E...,684.98
4,MATERIAL DE COPA E COZINHA,5322.77
5,MATERIAL DE LIMPEZA E PROD. DE,11195.70
6,MATERIAL P/ MANUT.DE BENS IMOV,16050.33
7,MATERIAL ELETRICO E ELETRONICO,30965.01
8,MATERIAL DE PROTECAO E SEGURANCA,11060.15
9,FERRAMENTAS,1889.38
